Convert practical salinity open boundary files to TEOS-10 reference salinity open boundary files

2016-03-12 : Couldn't get the ncatted to work (segmentation fault).  Went back to the PrepareSimpleTS-Johnstone and added TEOS10 calculation at the bottom of the that (made new file from scratch)

In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from salishsea_tools import nc_tools
from salishsea_tools import teos_tools
%matplotlib inline

Read in the original practical salinity file

In [2]:
practical_data = nc.Dataset('../../../NEMO-forcing/open_boundaries/north/SalishSea2_North_tra.nc')
practical_salinity = practical_data.variables['vosaline'][:]
print (practical_salinity.shape)

(2, 40, 1, 300)


Now we have to work around a bug in netcdf4 library.  We want to change the attributes of the vosaline 
variable, but this crashes.  So instead we go out to the command line and copy the file and change the vosaline attributes. Note ncatted is only available on some machines, e.g. Salish (these are for west file, last use of this notebook was North file)

cp ../../../NEMO-forcing/open_boundaries/west/SalishSea2_Masson_corrected.nc SalishSea_west_TEOS10.nc   
ncatted -O -a units,vosaline,m,c,'g/kg' SalishSea_west_TEOS10.nc   
ncatted -O -a long_name,vosaline,m,c,'Reference Salinity' SalishSea_west_TEOS10.nc   
ncatted -O -h -a history,global,a,c,'[2016-01-23] Converted to Reference Salinity' SalishSea_west_TEOS10.nc   
ncatted -O -h -a title,global,o,c,'Modified boundary conditions based on weekly climatolgy from Diane Masson converted to TEOS' SalishSea_west_TEOS10.nc   
ncatted -O -h -a source,global,a,c,'https://bitbucket.org/salishsea/tools/src/tip/I_ForcingFiles/OBC/TEOSfromPracticalOBC.ipynb' SalishSea_west_TEOS10.nc   
ncatted -O -h -a comment,global,a,c,'Converted to Reference Salinity (TEOS-10)' SalishSea_west_TEOS10.nc   
ncatted -O -h -a references,global,a,c,'https://bitbucket.org/salishsea/nemo-forcing/src/tip/open_boundaries/west/SalishSea_west_TEOS10.nc' SalishSea_west_TEOS10.nc

In [3]:
TEOS_data = nc.Dataset('SalishSea_north_TEOS10.nc', 'r+')
print (nc_tools.show_variable_attrs(TEOS_data))
print (nc_tools.show_dataset_attrs(TEOS_data))

RuntimeError: No such file or directory

In [ ]:
print (practical_salinity[1,20,0,40],teos_tools.psu_teos(practical_salinity[1,20,0,40]))

In [ ]:
ref_sal = np.zeros_like(practical_salinity)
ref_sal = teos_tools.psu_teos(practical_salinity)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15,5))
i = 1; k=0; l1=0; l2=30
ax[0].pcolormesh(practical_salinity[i,:,k,l1:l2])
ax[0].set_title('Practical Salinity')
ax[1].pcolormesh(ref_sal[i,:,k,l1:l2])
ax[1].set_title('Reference Salinity')
ax[2].pcolormesh(ref_sal[i,:,k,l1:l2]/practical_salinity[i,:,k,l1:l2])
ax[2].set_title('Ratio')
print (ref_sal[i,10,k,15]/practical_salinity[i,10,k,15])

In [ ]:
vosaline = TEOS_data.variables['vosaline']
vosaline[:] = ref_sal

In [ ]:
TEOS_data.close()